# METHODE 1
## Avec une seule carte en utilisant les commandes "classiques"

In [12]:
!make

Building application "ttn-sensors" for "b-l072z-lrwan1" with MCU "stm32".

"make" -C /home/jovyan/work/training/riot/RIOT/pkg/semtech-loramac
"make" -C /home/jovyan/work/training/riot/RIOT/build/pkg/semtech-loramac/src/boards/mcu -f /home/jovyan/work/training/riot/RIOT/pkg/semtech-loramac/Makefile.loramac_arch
"make" -C /home/jovyan/work/training/riot/RIOT/build/pkg/semtech-loramac/src/system/crypto -f /home/jovyan/work/training/riot/RIOT/pkg/semtech-loramac/Makefile.loramac_crypto
"make" -C /home/jovyan/work/training/riot/RIOT/build/pkg/semtech-loramac/src/mac -f /home/jovyan/work/training/riot/RIOT/pkg/semtech-loramac/Makefile.loramac_mac
"make" -C /home/jovyan/work/training/riot/RIOT/build/pkg/semtech-loramac/src/mac/region -f /home/jovyan/work/training/riot/RIOT/pkg/semtech-loramac/Makefile.loramac_region
"make" -C /home/jovyan/work/training/riot/RIOT/boards/b-l072z-lrwan1
"make" -C /home/jovyan/work/training/riot/RIOT/core
"make" -C /home/jovyan/work/training/riot/RIOT/cpu/stm32
"

In [8]:
!iotlab-experiment submit -n "ttn-sensors" -d 120 -l 1,archi=st-lrwan1:sx1276+site=saclay

{
    "id": 306674
}


In [9]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

Waiting that experiment 306674 gets in state Running
"Running"


In [10]:
!make IOTLAB_NODE=auto flash-only

In [11]:
!iotlab-experiment stop

# METHODE 2
## Pour utiliser plusieurs cartes sur différents sites

In [11]:
!make

In [11]:
# Code pour lancer deux cartes sur le site de Saclay et deux cartes sur le site de Lille
# Malheureusement les cartes de Lille n'ont pas de capteur de température donc rien n'est remonté
# Tout remonte sur une seule device TTN

import time
import os
number_exeperience = 2
number_exeperience = number_exeperience*2
id_experience = []
i = 0

status = []
while(1):
    if (i % 2) == 0:
        id_experience = !iotlab-experiment submit -n "Saclay" -d 120 -l 2,archi=st-lrwan1:sx1276+site=saclay
    else:
        id_experience = !iotlab-experiment submit -n "Lille" -d 120 -l 2,archi=st-lrwan1:sx1276+site=lille
    print("wait")
    status = !iotlab-experiment wait --timeout 30 --cancel-on-timeout
    time.sleep(30)
    !iotlab-experiment --jmespath="items[*].network_address | sort(@)" get --nodes
    print("etape1")
    time.sleep(5)
    print("etape2")
    !make BOARD=b-l072z-lrwan1 IOTLAB_NODES=2 -C ../ttn-sensors iotlab-flash
    time.sleep(30)
    print("etape3")
    !iotlab-experiment stop
    time.sleep(30)
    i = i + 1
    if i>number_exeperience:
        break

# METHODE 3
## En utilisant le testbed en ligne Fit-Iot Lab

Lancer une expérience à Saclay avec deux nodes "st-lrwan".
Sur une carte flasher avec ttn-sensors.elf, sur l'autre avec ttn-sensors-bis.elf.

Cela permet de se connecter sur deux devices différents sur TTN. 

Faire un reset de la carte jusqu'à être connecté.